# L2 Bridging and Queue Parameters

<div style="max-width:600px;margin-left: 0; margin-right: auto;">
    <img src="fig/3nets.drawio.png" alt="Drawing" style="width: 600px;"/>
</div>

The color of lines tells what network segment is used.

In [ ]:
from ns import ns

In [ ]:
import random
from typing import Union

ns.cppyy.cppdef("""
    EventImpl* MakeChangeIntervalEvent(void (*f)(Ptr<UdpEchoClient>), Ptr<UdpEchoClient> app)
    {
        return MakeEvent(f, app);
    }
""")

def change_interval_event(echo_client: Union[ns.UdpEchoClient, ns.Ptr]) -> None:
    try:
        echo_client = echo_client.__deref__()
    except AttributeError as e:
        # already a UdpEchoClient
        pass
    interval = random.randint(100,500) / 1000
    echo_client.SetAttribute("Interval", ns.core.TimeValue(ns.core.Seconds(interval)))
    event = ns.cppyy.gbl.MakeChangeIntervalEvent(change_interval_event, echo_client)
    ns.Simulator.Schedule(ns.Seconds(5), event)


## Network definition and Container creation

In [ ]:
networks = (1,2,3)
node_containers = []

bridge_device_containers = [ns.internet.NetDeviceContainer() for _ in range(len(networks))]
net_device_containers = [ns.internet.NetDeviceContainer() for _ in range(len(networks))]

switch_container = ns.network.NodeContainer()
switch_container.Create(len(networks))
router_node_container = ns.network.NodeContainer()
router_node_container.Create(1)

> Note that we are creating a container for a single node. This is because of the internet stack installation several steps later.

In [ ]:
for network in networks:
    node_container = ns.network.NodeContainer()
    node_container.Create(3)
    node_containers.append(node_container)

## Link creation

> Note that for bridged devices we cannot use Point-to-Point links.

In [ ]:
csma = ns.csma.CsmaHelper()

In [ ]:
for idx, node_container in enumerate(node_containers):
    for node_idx in range(node_container.GetN()):
        csma_node_pair = ns.network.NodeContainer()
        csma_node_pair.Add(node_container.Get(node_idx))
        csma_node_pair.Add(switch_container.Get(idx))
        devices = csma.Install(csma_node_pair)
        bridge_device_containers[idx].Add(devices.Get(1))
        net_device_containers[idx].Add(devices.Get(0))
    csma_node_pair = ns.network.NodeContainer()
    csma_node_pair.Add(switch_container.Get(idx))
    csma_node_pair.Add(router_node_container.Get(0))
    devices = csma.Install(csma_node_pair)
    bridge_device_containers[idx].Add(devices.Get(0))
    net_device_containers[idx].Add(devices.Get(1))

## Create bridges

In [ ]:
bridge_helper = ns.bridge.BridgeHelper()

In [ ]:
for idx, bridge_device_container in enumerate(bridge_device_containers):
    bridge_helper.Install(switch_container.Get(idx), bridge_device_container)

### Task

Check the number of devices on switches.

## Adding network stack

In [ ]:
internet = ns.internet.InternetStackHelper()

for node_container in node_containers:
    internet.Install(node_container)

# this step can cause a kernel reset if performed on nodes directly
internet.Install(router_node_container)

In [ ]:
addr = ns.internet.Ipv4AddressHelper()
addr.SetBase(ns.network.Ipv4Address(f"10.0.{networks[0]}.0"),
                ns.network.Ipv4Mask("255.255.255.0"))

ifaces = []

for idx, network_id in enumerate(networks):
    device_ifaces = addr.Assign(net_device_containers[idx])
    ifaces.append(device_ifaces)
    addr.NewNetwork()

In [ ]:
ifaces

In [ ]:
ifaces[0].GetN()

## Adding applications

- Servers: on each 3rd node and also on 2nd node of red and blue networks.
- Clients: on each 1st node and also on 2nd node of yellow network.

In [ ]:
local_servers = ns.network.NodeContainer()
local_clients = ns.network.NodeContainer()
remote_servers = ns.network.NodeContainer()
remote_clients = ns.network.NodeContainer()

for idx, network_id in enumerate(networks):
    local_servers.Add(node_containers[idx].Get(2))
    local_clients.Add(node_containers[idx].Get(0))
    if idx == 0:
        remote_clients.Add(node_containers[idx].Get(1))
    else:
        remote_servers.Add(node_containers[idx].Get(1))

servers = ns.network.NodeContainer()
servers.Add(local_servers)
servers.Add(remote_servers)

In [ ]:
ECHO_PORT = 9

In [ ]:
echo_srv_helper = ns.UdpEchoServerHelper(ECHO_PORT)
srv_apps = echo_srv_helper.Install(servers)
print(f"Servers: {srv_apps.GetN()}")

In [ ]:
from lib.utils import get_node_ips, create_echo_client_helper

In [ ]:
get_node_ips(servers.Get(0), verbose=True)

In [ ]:
echo_clients = ns.applications.ApplicationContainer()
for local_client_idx in range(local_clients.GetN()):
    local_server_node = local_servers.Get(local_client_idx)
    # always second interface, then first address
    local_server_address = get_node_ips(local_server_node)[1][0].GetAddress().ConvertTo()
    
    echo_client_helper = create_echo_client_helper(local_server_address, ECHO_PORT)
    echo_client_app = echo_client_helper.Install(local_clients.Get(local_client_idx))
    echo_clients.Add(echo_client_app)

remote_server_address = get_node_ips(remote_servers.Get(0))[1][0].GetAddress().ConvertTo()

echo_client_helper = create_echo_client_helper(remote_server_address, ECHO_PORT)
echo_client_app = echo_client_helper.Install(remote_clients.Get(0))
echo_clients.Add(echo_client_app)

remote_server_address = get_node_ips(remote_servers.Get(1))[1][0].GetAddress().ConvertTo()

echo_client_helper = create_echo_client_helper(remote_server_address, ECHO_PORT)
echo_client_app = echo_client_helper.Install(remote_clients.Get(0))
echo_clients.Add(echo_client_app)

### Task

Randomize the packet sending times.

## Install routing tables

In [ ]:
ns.internet.Ipv4GlobalRoutingHelper.PopulateRoutingTables()

## Schedule events

In [ ]:
# Schedule packet size change
ec = echo_clients.Get(0).GetObject[ns.UdpEchoClient]()
event = ns.cppyy.gbl.MakeChangeIntervalEvent(change_interval_event, ec)
ns.Simulator.Schedule(ns.Seconds(5), event)

## Timing the simulation

In [ ]:
srv_apps.Start(ns.core.Seconds(1.0))
srv_apps.Stop(ns.core.Seconds(30.0))
echo_clients.Start(ns.core.Seconds(2.0)) # has to be later than server
echo_clients.Stop(ns.core.Seconds(30.0))

## Enabling logging for applications

In [ ]:
ns.LogComponentEnable("UdpEchoClientApplication", ns.core.LOG_LEVEL_INFO)
ns.LogComponentEnable("UdpEchoServerApplication", ns.core.LOG_LEVEL_INFO)

## Running the simulation

In [ ]:
ns.Simulator.Stop(ns.Seconds(35))
ns.Simulator.Run()
ns.Simulator.Destroy()